In [1]:
import os
cwd = os.getcwd()

# protection against running this cell multiple times
assert os.path.dirname(cwd).split('/')[-1] == 'master-thesis','Oops, directory already changed previously as indended. Ignoring...'

# change working directory (if assert passed)
new_cwd = os.path.dirname(cwd) # parent directory
os.chdir(new_cwd)

In [1]:
import os
from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import RNN, train_one_epoch, train_one_epochV2, eval_model

import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np

from config import Settings; settings = Settings()

import wandb

# Parameters

In [2]:
dataset_id = 'diag_only'

# model
grouping = 'ccs'
batch_size=64

# Reproducibility

In [3]:
# Reproducibility
seed = settings.random_seed

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Create dataset

In [4]:
dataset_folder = os.path.join(settings.data_base,settings.model_ready_dataset_folder,dataset_id)
print('dataset at',dataset_folder)

dataset at data/model_ready_dataset/diag_only


In [5]:
dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset.json'),grouping)

train_dataset = DiagnosesDataset(os.path.join(dataset_folder,'train_subset.json'),grouping)
val_dataset = DiagnosesDataset(os.path.join(dataset_folder,'val_subset.json'),grouping)
test_dataset = DiagnosesDataset(os.path.join(dataset_folder,'test_subset.json'),grouping)


len(train_dataset)
len(val_dataset)
len(test_dataset)


train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

5249

1125

1125

# wandb

In [6]:
wandb.login()

wandb: Currently logged in as: snovaisg (use `wandb login --relogin` to force relogin)


True

## Define data and train conditions

In [7]:
n_labels = input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

## Define search space

In [8]:
hyperparameters = {
    'hidden_size':[25,50,100,150], #-1 is to have the same as input size
    'num_layers':[1,2],
    'lr':[0.01,0.02,0.03],
    'model':['rnn','gru','lstm']
    
}
meta_parameters = {
    'epochs':15
}

params = ParameterGrid(hyperparameters)
print(f'params:',len(params))

#random_params = ParameterSampler(params.param_grid,n_iter=len(params)-1,random_state=231)
#next(iter(random_params))

params: 72


## Run models

In [9]:
for param_set in params:
    config = {**param_set, 
              **meta_parameters}
    
    wandb.init(
        project="thesis_basic_deterministic_model_tunning", 
        config=config
    )
    
    model = RNN(input_size=input_size,
              hidden_size=config['hidden_size'],
              num_layers=config['num_layers'],
              n_labels=n_labels,
              model=config['model'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    for epoch in range(1,config['epochs']+1):
        loss = train_one_epochV2(model,train_dataloader,epoch,criterion,optimizer)
        wandb.log({"loss":loss})
    
    # eval model
    train_results = eval_model(model,train_dataloader,dataset, criterion, epoch, 'train_last')
    val_results = eval_model(model,val_dataloader,dataset, criterion, epoch, 'validation')
    
    # log
    wandb.log({'train_loss':train_results['loss'],
               'train_recall@30':train_results['last adm']['recall30']['mean'],
               'val_loss':val_results['loss'],
               'recall@10':val_results['last adm']['recall10']['mean'],
               'recall@20':val_results['last adm']['recall20']['mean'],
               'recall@30':val_results['last adm']['recall30']['mean']
              })

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  

loss,0.12129
_runtime,23
_timestamp,1646743355
_step,15
train_loss,0.1201
train_recall@30,0.65726
val_loss,0.12063
recall@10,0.37645
recall@20,0.54677
recall@30,0.6594


loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▃▃▃▄▄▅▅▅▅▆▆█
_timestamp,▁▁▂▂▃▃▃▄▄▅▅▅▅▆▆█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


loss,0.07908
_runtime,27
_timestamp,1646745971
_step,15
train_loss,0.07256
train_recall@30,0.88786
val_loss,0.13537
recall@10,0.41297
recall@20,0.5636
recall@30,0.66002


loss,█▅▄▄▃▃▃▂▂▂▂▁▁▁▁
_runtime,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▇█
_timestamp,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▇█
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train_loss,▁
train_recall@30,▁
val_loss,▁
recall@10,▁
recall@20,▁
recall@30,▁


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error resolved after 0:01:23.441569, resuming normal operation.


# best model

- lr: 0.03
- model_type = 'lstm'
- num_layers = 1
- hidden_size = 50
- epochs: 15